In [1]:
from bs4 import BeautifulSoup
import requests
from urllib.parse import urljoin
import re
from tqdm import tqdm

In [2]:
def link2soup(url):
    response = requests.get(url)
    return BeautifulSoup(response.text, 'html.parser')

In [3]:
# very rudimentary class for data storage

class CreditCard:
    def __init__(self, name, annual_fee, welcome_amount, apr, rewards):
        self.name = name
        self.annual_fee = annual_fee
        self.welcome_amount = welcome_amount
        self.apr = apr
        self.rewards = rewards
        
    def __repr__(self):
        return f"CreditCard(name = {self.name},\n annual_fee = {self.annual_fee},\n welcome_amount = {self.welcome_amount},\n apr = {self.apr},\n rewards = {self.rewards})"

In [26]:
soup = link2soup("https://www.capitalone.com/credit-cards/compare/")
card_container = soup.select("card-product-all-cards-list-item")
card_container = card_container[0:-1] #have some weird formatting with an extra block

In [30]:
cards = []

for card in card_container:
    name = re.sub(r'[^ -~]', '', card.select("a.product-name.ng-star-inserted")[0].text)
    try:
        welcome_amount = card.select("div.short-desc.ng-star-inserted")[0].text
    except IndexError:
        welcome_amount = None
    rewards = re.sub("Rewards", "", card.select("div.feature.ng-star-inserted")[0].text)
    annual_fee = re.sub("Annual Fee", "", card.select("div.feature.ng-star-inserted")[1].text)
    apr = re.sub("Purchase Rate", "", card.select("div.feature.ng-star-inserted")[2].text)
    card_obj = CreditCard(name, annual_fee, welcome_amount, apr, rewards)
    cards.append(card_obj)

In [31]:
for card in cards:
    print(card)

CreditCard(name = Platinum Mastercard,
 annual_fee = $0,
 welcome_amount = No annual fee. Flexible due dates. See if youâre approved in seconds.,
 apr = 29.49% variable APR,
 rewards = Get the credit you need with no annual fee.)
CreditCard(name = Venture X Rewards,
 annual_fee = $395,
 welcome_amount = Earn 75,000 bonus miles once you spend $4,000 on purchases within the first 3 months from account opening
1
,
 apr = 19.99% - 28.99% variable APR,
 rewards = Earn unlimited 10X miles on hotels and rental cars booked through Capital One Travel and 5X miles on flights and vacation rentals booked through Capital One Travel, plus unlimited 2X miles on all other purchases)
CreditCard(name = Venture Rewards,
 annual_fee = $95,
 welcome_amount = Earn 75,000 bonus miles once you spend $4,000 on purchases within the first 3 months from account opening
2
,
 apr = 19.99% - 28.99% variable APR,
 rewards = Earn unlimited 2 miles per dollar on every purchase. Earn unlimited 5 miles per dollar on ho

In [17]:
card_container[0].select("div.feature.ng-star-inserted")[2]

<div _ngcontent-web-shell-c770310391="" class="feature ng-star-inserted"><div _ngcontent-web-shell-c770310391="" class="feature-header">Purchase Rate</div><div _ngcontent-web-shell-c770310391="">29.49% variable APR</div></div>